In [15]:
import pandas as pd
import sqlite3

In [20]:
query = '''
SELECT
    u.name,
    u.city,
    u.street,
    u.favourite_genres,
    a.user_id,
    a.year,
    a.month,
    CAST(COALESCE(aaa.number_of_advertisements, 0) AS INTEGER) as number_of_advertisements,
    CAST(COALESCE(aa.number_of_skips, 0) AS INTEGER) as number_of_skips,
    CAST(COALESCE(b.number_of_likes, 0) AS INTEGER) as number_of_likes,
    c.total_tracks_duration_ms,
    d.number_of_different_artists,
    e.average_release_date,
    e.average_duration_ms,
    e.explicit_tracks_ratio,
    e.average_popularity,
    e.average_acousticness,
    e.average_danceability,
    e.average_energy,
    e.average_instrumentalness,
    e.average_liveness,
    e.average_loudness,
    e.average_speechiness,
    e.average_tempo,
    e.average_valence,
    f.premium_purchased,
    g.premium_purchased as premium_purchased_this_month,
    CAST(COALESCE(h.premium_purchased_next_month, 0) AS INTEGER) AS premium_purchased_next_month
FROM
    (SELECT
        user_id,
        strftime('%Y', timestamp) as year,
        strftime('%m', timestamp) as month
    FROM sessions
    GROUP BY user_id, year, month) a
LEFT JOIN
    (SELECT
            user_id,
            strftime('%Y', timestamp) as year,
            strftime('%m', timestamp) as month,
            COUNT(*) as number_of_advertisements
        FROM sessions
        WHERE event_type = 'advertisement'
        GROUP BY user_id, year, month) aaa
ON 
    a.user_id = aaa.user_id AND a.year = aaa.year AND a.month = aaa.month
LEFT JOIN
    (SELECT
        user_id,
        strftime('%Y', timestamp) as year,
        strftime('%m', timestamp) as month,
        COUNT(*) as number_of_skips
    FROM sessions
    WHERE event_type = 'skip'
    GROUP BY user_id, year, month) aa
ON 
    a.user_id = aa.user_id AND a.year = aa.year AND a.month = aa.month
LEFT JOIN
    (SELECT
        user_id,
        strftime('%Y', timestamp) as year,
        strftime('%m', timestamp) as month,
        COUNT(*) as number_of_likes
    FROM sessions
    WHERE event_type = 'like'
    GROUP BY user_id, year, month) b
ON
    a.user_id = b.user_id AND a.year = b.year AND a.month = b.month
LEFT JOIN
    (SELECT
        user_id,
        strftime('%Y', timestamp) as year,
        strftime('%m', timestamp) as month,
        SUM(tracks.duration_ms) as total_tracks_duration_ms
    FROM
        sessions
    JOIN
        tracks ON sessions.track_id = tracks.id
    WHERE
        event_type = 'play'
    GROUP BY
        year, month, user_id) c
ON
    a.user_id = c.user_id AND a.year = c.year AND a.month = c.month
LEFT JOIN
    (SELECT
        user_id,
        strftime('%Y', timestamp) as year,
        strftime('%m', timestamp) as month,
        COUNT(DISTINCT artists.id) as number_of_different_artists
    FROM
        sessions
    JOIN
        tracks ON sessions.track_id = tracks.id
    JOIN
        artists ON tracks.id_artist = artists.id
    WHERE
        event_type = 'play'
    GROUP BY
        year, month, user_id) d
ON
    a.user_id = d.user_id AND a.year = d.year AND a.month = d.month
LEFT JOIN
    (SELECT
        strftime('%Y', s.timestamp) AS year,
        strftime('%m', s.timestamp) AS month,
        s.user_id,
        AVG(strftime('%s', t.release_date)) AS average_release_date,
        AVG(t.duration_ms) AS average_duration_ms,
        AVG(CAST(t.explicit AS FLOAT)) AS explicit_tracks_ratio,
        AVG(t.popularity) AS average_popularity,
        AVG(t.acousticness) AS average_acousticness,
        AVG(t.danceability) AS average_danceability,
        AVG(t.energy) AS average_energy,
        AVG(t.instrumentalness) AS average_instrumentalness,
        AVG(t.liveness) AS average_liveness,
        AVG(t.loudness) AS average_loudness,
        AVG(t.speechiness) AS average_speechiness,
        AVG(t.tempo) AS average_tempo,
        AVG(t.valence) AS average_valence
    FROM
        sessions s
    JOIN
        tracks t ON s.track_id = t.id
    WHERE
        s.event_type = 'play'
    GROUP BY
        year, month, s.user_id) e
ON
    a.user_id = e.user_id AND a.year = e.year AND a.month = e.month
LEFT JOIN
    (SELECT
        strftime('%Y', s.timestamp) AS year,
        strftime('%m', s.timestamp) AS month,
        s.user_id,
        MAX(u.premium_user) AS premium_purchased
    FROM sessions s
    JOIN users u ON s.user_id = u.user_id
    GROUP BY year, month, s.user_id) f
ON
    a.user_id = f.user_id AND a.year = f.year AND a.month = f.month
LEFT JOIN
    (SELECT
        strftime('%Y', s.timestamp) AS year,
        strftime('%m', s.timestamp) AS month,
        s.user_id,
        MAX(CASE WHEN s.event_type = 'buy_premium' THEN 1 ELSE 0 END) AS premium_purchased
    FROM sessions s
    GROUP BY year, month, s.user_id) g
ON
    a.user_id = g.user_id AND a.year = g.year AND a.month = g.month
LEFT JOIN
    (SELECT
        strftime('%Y', s.timestamp, '-1 month') AS prev_year,
        strftime('%m', s.timestamp, '-1 month') AS prev_month,
        strftime('%Y', s.timestamp ) AS current_year,
        strftime('%m', s.timestamp) AS current_month,
        strftime('%Y', s.timestamp, '+1 month') AS year,
        strftime('%m', s.timestamp, '+1 month') AS month,
        s.user_id,
        MAX(CASE WHEN s.event_type = 'buy_premium' THEN 1 ELSE 0 END) AS premium_purchased_next_month
    FROM sessions s
    GROUP BY s.user_id, year, month) h
ON
    a.user_id = h.user_id AND a.year = h.prev_year AND a.month = h.prev_month
JOIN
    users u
ON
    a.user_id = u.user_id
ORDER BY
    a.user_id, a.year, a.month;

'''

In [21]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('/home/irek/Desktop/ium/database.db')
data = pd.read_sql(query, conn)
conn.close()

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147166 entries, 0 to 147165
Data columns (total 28 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   name                          147166 non-null  object 
 1   city                          147166 non-null  object 
 2   street                        147166 non-null  object 
 3   favourite_genres              147166 non-null  object 
 4   user_id                       147166 non-null  int64  
 5   year                          147166 non-null  object 
 6   month                         147166 non-null  object 
 7   number_of_advertisements      147166 non-null  int64  
 8   number_of_skips               147166 non-null  int64  
 9   number_of_likes               147166 non-null  int64  
 10  total_tracks_duration_ms      147165 non-null  float64
 11  number_of_different_artists   147165 non-null  float64
 12  average_release_date          147164 non-nul

In [15]:
import sqlite3
import pandas as pd
query = '''
SElect         strftime('%Y', timestamp ) AS year, count(*)  from sessions group by year;
'''
conn = sqlite3.connect('database.db')
data = pd.read_sql(query, conn)
conn.close()
data.head(50)

,year,count(*)
0,2023,10148165


In [24]:
data.to_csv('../data/final_dataset.csv', index=False)